# About
This notebook explores the dataset provided in the __[Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction)__ competition. The goal of the competition is to develop a model that will accept or reject trading opportunities based on "real-time" market data. The market data is presented in the form of a time series and at each point in time the model must accept/reject a new trade using only the data up to that time. At each point in time, we are given a vector

` date, weight, feature_0, feature_1, ..., feature_129, ts_id`

The times of the time series, `ts_id`, are grouped into days that are recorded in `date`. The features `feature_0, ...` consist of various market data has been "anonymized" (presumably to protect Jane Street's business model), so we don't really know what they represent. `weight` is the size of our investment in the proposed trade. The model should output 0 to reject the trade and 1 to accept it. Each trade results in a profit/loss response variable `resp`.

For the $i$-th day, the model is given the score
$$p_i = \sum_j \mbox{weight}_{ij} \cdot \mbox{resp}_{ij}$$
where the sum ranges over all accepted trades for the day. The final score of the model over all days is then
$$u = \min(\max(t, 0), 6) \cdot \sum_i p_i $$
with
$$t = \frac{\sum_i p_i}{\sqrt{\sum_i p_i^2}} \sqrt{\frac{250}{\mbox{total number of days}}}.$$

To train our model, we are given a file `train.csv` which contains 500 days worth of data or about 2 million data points (times) overall. For each point in time we are given the variables above along with the response variable `resp`. In addition, we are given 4 further response variables `resp_1, ..., resp_4`, which (according to the organizers) represent the profits achieved by the trade over different time scales. We are also given a boolean matrix in `features.csv`, which is explained below. Finally, we are given an example test set in `example_test.csv`, which contains some time series that are representative of what the model will see in production.

# Summary
This notebook consists of three sections. First, we examine correlations among the features. Then we train some simplistic models that ignore the time series structure and fit regressors for the response variable based only on the features at the same point in time. We attempt to analyze the predictive power of the most important features in these regression models. In the final section, we consider the time series structure of the data and try to determine the predictive power of historical data for the future.

# Conclusions
Most of the preliminary analysis indicates that we have a very noisy dataset that is prone to overfitting. It seems we should place an emphasis on a strong denoising / feature selection process and an ensemble of simple models on the new features. The score above emphasizes steady and consistent gains, so our model should have a low variance even if this costs some bias.

We start by running some setup code and parsing the CSV files provided in the dataset. We fill in missing values with the mean of that feature over all samples.

In [ ]:
%%capture
%pip install datatable
%pip install "seaborn>=0.11.0"

import math
import os

import numpy as np

import datatable as dt
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.stattools import acf, ccf, adfuller

# make the labels legible on plots
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels

comp_folder = os.path.join(os.pardir, "input", "jane-street-market-prediction")

train_df = dt.fread(os.path.join(comp_folder, "train.csv")).to_pandas()
train_df.set_index("ts_id", inplace=True)
test_df = pd.read_csv(os.path.join(comp_folder, "example_test.csv"), index_col="ts_id")
feat_df = pd.read_csv(os.path.join(comp_folder, "features.csv"), index_col="feature")

train_df.fillna(train_df.mean(), inplace=True)
test_df.fillna(test_df.mean(), inplace=True)

# list of columns with actual features
feat_cols = [c for c in train_df.columns if "feature" in c]

# keep a collection of any features which may be important
# as we go through the notebook
all_best_cols = []

# Feature Correlations
`features.csv` contains a boolean matrix whose $i,j$ entry is true if tag $i$ applies to feature $j$. A hypothetical example from the organizers is that the a following de-anonymized features and tags

| feature | meaning  |
|---:|:-------------|
| feature_0 | volatility of this stock in past 30 days |
| feature_1 | volume of this stock in past 30 days |
| feature_2 | volume of this stock in past 10 days |

| tag | meaning |
|---:|:-------------|
| tag_0 | some metric on the past 30 days  | False | 
| tag_1 | volume of this stock  | True  | 

would result in the following matrix:

|  | tag_0  | tag_1 |
|---:|:-------------|:-----------|
| feature_0 | True  | False | 
| feature_1 | True  | True  | 
| feature_2 | False | True  |

We plot the full matrix below.

In [ ]:
options = {"vmin": 0, "vmax": 1,  "linewidths": .5, "square": True,
           "cbar": False, "cmap": ["white", "black"], "linecolor": "black"
          }

plt.figure(figsize=(20, 8))
ax = sns.heatmap(feat_df.to_numpy().T, **options)

ax.set_xlabel("Feature")    
ax.set_ylabel("Tag")
ax.set(xticklabels=[])
ax.tick_params(bottom=False)
ax.set(yticklabels=[])
ax.tick_params(left=False)
plt.show()

We check for correlations among the features. The image below is annotated and has a reasonable resolution in standalone viewers.

In [ ]:
corr = train_df[feat_cols].corr(method="pearson")

mask = np.triu(np.ones_like(corr, dtype=np.bool))
options = {"vmin": -1, "vmax": 1,  "linewidths": .5, "square": True,
           "cbar": False, "cmap": "vlag", "mask": mask, "annot": True,
           "linecolor": "black"
          }

plt.figure(figsize=(80, 80))
ax = sns.heatmap(corr.to_numpy(), **options)
ax.set_xlabel("Feature")
ax.set_ylabel("Feature")
plt.show()

The training data set has 4 addtional response variables along with `resp` - the plots below generally show strong positive correlations among them.

In [ ]:
sns.pairplot(train_df[["resp", "resp_1", "resp_2", "resp_3", "resp_4"]], 
             diag_kind="kde", plot_kws={"s": 3})
plt.show()

We perform a __[principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis)__ and determine how much of the total variance is accounted for by each component.

In [ ]:
ss = StandardScaler()
pca = PCA()

scaled = ss.fit_transform(train_df[feat_cols])
pca.fit(scaled)

var_perc = pd.DataFrame(100 * pca.explained_variance_ratio_, columns=["y"])

plt.figure(figsize=(20, 8))
ax = sns.barplot(x=var_perc.index, y=var_perc.y)
ax.set_xlabel("Component")    
ax.set_ylabel("% of Total Variance")
ax.set(xticklabels=[])
ax.tick_params(bottom=False)
plt.show()

rank = 1 + np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.95)
print(f"{rank} components account for 95% of the total variance.")

# Predictive Power
To limit the computational power required, the remainder of this notebook only considers a subset of the data. More precisely, we restrict ourselves to the first time series on day 0.

In [ ]:
oneday = train_df.loc[train_df["date"] == 0]

We compute the __[mutual information](https://en.wikipedia.org/wiki/Mutual_information)__ of each feature with the response. Then we plot the three features with the highest mutual information against the response.

In [ ]:
mi = mutual_info_regression(oneday[feat_cols], oneday["resp"], random_state=42)

# 3 features with highest mutual information
best = [f"feature_{ix}" for ix in np.argsort(-mi)[:3]]

# add features to list of interesting features
all_best_cols = all_best_cols + best

mi_df = pd.DataFrame(mi, columns=["y"])

plt.figure(figsize=(20, 8))
ax = sns.barplot(x=mi_df.index, y=mi_df.y)
ax.set_xlabel("Feature")    
ax.set_ylabel("Mutual Information with resp")
ax.set(xticklabels=[])
ax.tick_params(bottom=False)
plt.show()

fig, axs = plt.subplots(1, 3, figsize=(20, 4))
flag = False
for (col, ax) in zip(best, axs):
    sns.scatterplot(x=col, y="resp", data=oneday, ax=ax, s=3)
    if flag:
        ax.set_ylabel("")
        ax.set(yticklabels=[])
        ax.tick_params(left=False)
    flag = True

plt.show()

We train a __[random forest](https://en.wikipedia.org/wiki/Random_forest)__ model and compute the __[importance](https://en.wikipedia.org/wiki/Random_forest#Properties)__ of each feature. Then we plot the three most important features against the response.

In [ ]:
rf = RandomForestRegressor(max_features="auto", random_state=42)
rf.fit(oneday[feat_cols], oneday["resp"])

# 3 features with highest importance
best = [f"feature_{ix}" for ix in np.argsort(-rf.feature_importances_)[:3]]

# add features to list of interesting features
all_best_cols = all_best_cols + best

imp_df = pd.DataFrame(rf.feature_importances_, columns=["y"])

plt.figure(figsize=(20, 8))
ax = sns.barplot(x=imp_df.index, y=imp_df.y)
ax.set_xlabel("Feature")    
ax.set_ylabel("Importance")
ax.set(xticklabels=[])
ax.tick_params(bottom=False)
plt.show()

fig, axs = plt.subplots(1, 3, figsize=(20, 4))
flag = False
for (col, ax) in zip(best, axs):
    sns.scatterplot(x=col, y="resp", data=oneday, ax=ax, s=3)
    if flag:
        ax.set_ylabel("")
        ax.set(yticklabels=[])
        ax.tick_params(left=False)
    flag = True

plt.show()

We perform a __[partial least squares](https://en.wikipedia.org/wiki/Partial_least_squares_regression)__ analysis and record the Variable Importance in Projection (VIP) scores for each feature. Then we plot the three most important features against the response. The VIP score implementation is taken from __[this](https://github.com/KevinMMendez/scikit-learn/commit/e532ac94dac52d6f7f1021970e887b83442f11e9)__ version of code presented in __[this](https://github.com/scikit-learn/scikit-learn/issues/7050)__ issue.

In [ ]:
ss = StandardScaler()
pls = PLSRegression(n_components=len(feat_cols))

X = ss.fit_transform(oneday[feat_cols])
y = oneday["resp"]
pls.fit(X, y)

# compute VIP scores
T = pls.x_scores_
W = pls.x_weights_
Q = pls.y_loadings_
w0, w1 = W.shape
s = np.sum(T ** 2, axis=0) * np.sum(Q ** 2, axis=0)
s_sum = np.sum(s, axis=0)
w_norm = np.array([(W[:, i] / np.linalg.norm(W[:, i])) for i in range(w1)])
vip = np.sqrt(w0 * np.sum(s * w_norm.T ** 2, axis=1) / s_sum)

# 3 features with highest VIP
best = [f"feature_{ix}" for ix in np.argsort(-vip)[:3]]

# add features to list of interesting features
all_best_cols = all_best_cols + best

vip_df = pd.DataFrame(vip, columns=["y"])

plt.figure(figsize=(20, 8))
ax = sns.barplot(x=vip_df.index, y=vip_df.y)
ax.set_xlabel("Feature")    
ax.set_ylabel("Variable Importance in Projection")
ax.set(xticklabels=[])
ax.tick_params(bottom=False)
plt.show()

fig, axs = plt.subplots(1, 3, figsize=(20, 4))
flag = False
for (col, ax) in zip(best, axs):
    sns.scatterplot(x=col, y="resp", data=oneday, ax=ax, s=3)
    if flag:
        ax.set_ylabel("")
        ax.set(yticklabels=[])
        ax.tick_params(left=False)
    flag = True

plt.show()

# Time Series Analysis
The dataset has a temporal structure. For each value of the `date` column, we have a sequence of samples indexed by a "time" `ts_id`. Not all of the time series have the same length, but all of them have at least 10000 entries.

In [ ]:
plt.figure(figsize=(8, 5))
ax = sns.histplot(data=train_df, x="date", edgecolor="black")

ax.set_xlabel("Date")    
ax.set_ylabel("Number of Timestamps")
ax.set(xticklabels=[])
ax.tick_params(bottom=False)
plt.show()

The __[augmented Dickey-Fuller test](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test)__ rejects the hypothesis of a unit root and the response looks like a stationary process. The __[autocorrelation function](https://en.wikipedia.org/wiki/Autocorrelation)__ decays rapidly and high frequencies are well-represented in the Fourier spectrum.

In [ ]:
adf = adfuller(oneday["resp"])   
print(f"ADF statistic: {adf[0]} \t p-value: {adf[1]}")

fig, axs = plt.subplots(1, 3, figsize=(20, 5))

sns.lineplot(data=oneday["resp"], ax=axs[0])
axs[0].set_xlabel("Time")    
axs[0].set_ylabel("resp")

auto_cor = acf(oneday["resp"], fft=True)
sns.lineplot(data=auto_cor, ax=axs[1])
axs[1].set_xlabel("Lag")    
axs[1].set_ylabel("Autocorrelation")

ft_abs = np.abs(np.fft.rfft(oneday["resp"]))
sns.lineplot(data=ft_abs, ax=axs[2])
axs[2].set_xlabel("Frequency")    
axs[2].set_ylabel("Amplitude")

plt.show()

We plot the __[cross-correlation functions](https://en.wikipedia.org/wiki/Cross-correlation)__ between the significant features identified in the previous section and the response.

In [ ]:
all_best_cols = list(set(all_best_cols))

nrows = math.ceil(len(all_best_cols) / 3)
fig, axs = plt.subplots(nrows, 3, figsize=(20, 5 * nrows))

for (col, ax) in zip(all_best_cols, axs.flatten()):
    cross_cor = ccf(oneday[col], oneday["resp"])[:40]
    sns.lineplot(data=cross_cor, ax=ax)             
    ax.set_title(col)

if nrows > 1:
    for ax in axs[:, 0]:
        ax.set_ylabel("Cross-correlation")

    for ax in axs[nrows - 1, :]:
        ax.set_xlabel("Lag")

else:
    axs[0].set_ylabel("Cross-correlation")
    for ax in axs:
        ax.set_xlabel("Lag")

plt.show()

Are there correlations between the days? (unlikely, given the decay of correlations).